<div class='alert alert-info' style='text-align:center'><h1>Export Images By Plane</h1>
- yet another MR processing notebook -</div>

#### This notebook exports a series from each study in the training set into the same plane.
- Since all series are not in the same plane, it makes sense to export them into the same plane.
- Slices that are not in the specified plane already are reformated with SimpleITK.
- Images are exported as JPG, ignoring empty slices (all black).
- A zip file is created that contains a single series from each test study, in the plane you choose.

In [ ]:
import os
import sys 
import numpy as np
import nibabel as nib
import SimpleITK as sitk
import matplotlib.pyplot as plt
import pydicom
import cv2
import shutil
from IPython.display import FileLink
from PIL import Image

In [ ]:
# Specify a series and plane to export
PLANE = 'axial'
SERIES = 'T1w'

In [ ]:
# Get a list of study directories
train_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/'
train_dirs = os.listdir(train_path)

if not os.path.exists(f'/kaggle/working/train'):
    os.mkdir('/kaggle/working/train')

In [ ]:
# This function gives a basic plane from the ImageOrientationPatient tag. It doesn't account for obliqueness. But we don't need to care about it.
# Will return 'unknown' if the image isn't exactly orthogonal.
def get_image_plane(loc):
    row_x = round(loc[0])
    row_y = round(loc[1])
    row_z = round(loc[2])
    col_x = round(loc[3])
    col_y = round(loc[4])
    col_z = round(loc[5])
    if (row_x, row_y, col_x, col_y) == (1,0,0,0):
        return "Coronal"
    if (row_x, row_y, col_x, col_y) == (0,1,0,0):
        return "Sagittal"
    if (row_x, row_y, col_x, col_y) == (1,0,0,1):
        return "Axial"
    return "Unknown"

In [ ]:
# Resample function. Original idea from -> https://www.kaggle.com/boojum/connecting-voxel-spaces

def resample(image, ref_image):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_image)
    resampler.SetInterpolator(sitk.sitkLinear) 
    resampler.SetTransform(sitk.AffineTransform(image.GetDimension()))
    resampler.SetOutputSpacing(ref_image.GetSpacing())
    resampler.SetSize(ref_image.GetSize())
    resampler.SetOutputDirection(ref_image.GetDirection())
    resampler.SetOutputOrigin(ref_image.GetOrigin())
    resampler.SetDefaultPixelValue(image.GetPixelIDValue())
    resamped_image = resampler.Execute(image)
    return resamped_image

In [ ]:
# Setup a simpleITK reader and load a volume as a reference plane. Choose a series that's in the plane you want to export the target series to.
# We'll use the following series as reference planes since we already know their planes.

# train_dirs[0]/T2w = sagittal plane
# train_dirs[0]/T1w = axial plane
# train_dirs[5]/T1wCE = coronal plane

if PLANE == 'axial':
    train_dir = train_dirs[0]
    train_series = "T1w"
if PLANE == 'sagittal':
    train_dir = train_dirs[0]
    train_series = "T2w"   
if PLANE == 'coronal':
    train_dir = train_dirs[5]
    train_series = "T1wCE"

reader = sitk.ImageSeriesReader()
reader.LoadPrivateTagsOn()

ref_files = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dir}/{train_series}')
reader.SetFileNames(ref_files)
ref_set = reader.Execute()
     
ref_plane = get_image_plane(ref_set.GetDirection())
ref_array = sitk.GetArrayFromImage(ref_set)
print("Reference Image Plane: " + ref_plane)

In [ ]:
# Get a series into an array
def get_target_array(study, series):

    target_files = reader.GetGDCMSeriesFileNames(f'{train_path}/{study}/{series}')
    reader.SetFileNames(target_files)
    target_set = reader.Execute()

    target_plane = get_image_plane(target_set.GetDirection())

    # Only resample images that aren't already in the reference plane, otherwise just export as-is.
    if ref_plane != target_plane:
        target_new = resample(target_set, ref_set)
    else:
        target_new = target_set

    return sitk.GetArrayFromImage(target_new)

In [ ]:
# Crunch pixels down to 8 bit
def normalize_pixels(pixels):
    pixels = pixels - np.min(pixels)
    pixels = pixels / np.max(pixels)
    pixels = (pixels * 255).astype(np.uint8)
    return pixels

In [ ]:
def get_images(study, series):
    count = 0

    # Get the images in the target series
    target_array = get_target_array(study, series) 

    # Make dirs to save images to
    if not os.path.exists(f'/kaggle/working/train/{study}'):
        os.mkdir(f'/kaggle/working/train/{study}')
    if not os.path.exists(f'/kaggle/working/train/{study}/{series}'):
        os.mkdir(f'/kaggle/working/train/{study}/{series}')

    # iterate through each image in the array
    for i in range(0,target_array.shape[0]):

        if len(target_array) > 0:
            target = target_array[i,:,:]
            pixels = normalize_pixels(target)

            # Just check the center of the image to ignore the grey border we get with matrices that aren't the same shape (256 vs 194 etc)
            w = pixels.shape[0] - 100
            h = pixels.shape[1] - 100
            pix_mean = np.mean(pixels[100:w,100:h])

            # Export only images that have 'some' pixels. Not sure of the exact threshold here. 30 seems reasonable.
            # Maybe this should be a count of pixels that have any value other than the background value (~0) instead of mean since it will be different for each series type
            if pix_mean > 20:
                filename = f'/kaggle/working/train/{study}/{series}/{i}.jpg'
                cv2.imwrite(filename, pixels)
                count += 1

In [ ]:
# Iterate through each study directory
for study in train_dirs:
    get_images(study, SERIES)

In [ ]:
# Zip it up
shutil.make_archive(f'{SERIES}_{PLANE}', 'zip', '/kaggle/working/train')

In [ ]:
# Create a link to download the zip
os.chdir(r'/kaggle/working')
%cd /kaggle/working
FileLink(f'{SERIES}_{PLANE}.zip')

#### Some of my other MR notebooks
- Tumor Object Detection -> https://www.kaggle.com/davidbroberts/brain-tumor-object-detection
- Determining MR Slice Orientation -> https://www.kaggle.com/davidbroberts/determining-mr-slice-orientation
- Determining DICOM Image Order -> https://www.kaggle.com/davidbroberts/determining-dicom-image-order/
- Determining MR image planes -> https://www.kaggle.com/davidbroberts/determining-mr-image-planes
- Reference Lines on MR images -> https://www.kaggle.com/davidbroberts/mr-reference-lines
- Manual VOI LUT on MR images -> https://www.kaggle.com/davidbroberts/manual-voi-lut-on-mr-images
- Standardizing MR images -> https://www.kaggle.com/davidbroberts/standardizing-mr-images